In [119]:
from ipstartup import *
from pipe import *
import spacy
from spacy import displacy
#log.setLevel(logging.DEBUG)
os.chdir("/mnt/d/data1/reports")
nlp = spacy.load("en_core_web_sm")

# TEXT. poor OCR in some places including dates
f1 = "0a4b2b4d-8131-446f-acda-e4c2ec109663.pdf"
# IMAGE. OCRed in chat
f2 = "c588477d-366b-4613-abfc-c4c052e49c74.pdf"
# TEXT
f3 = "0a5351eb-6061-49cb-8d3a-2ad75a924ff7.pdf"
# text only includes the sustainability part of the report
f4 = "0dbd0328-6b3f-46fd-86dd-5dd09923adb4.pdf"

[cellevents:43:INFO]:starting (time=Jun-19 18:51 MainProcess)


time: 1.08 s


# load text pdftotext

In [ ]:
# FAILS WITH COLUMN BLENDING

In [17]:

with open(f"/mnt/d/data1/reports/{f}", "rb") as f:
    pdf = pdftotext.PDF(f)
print(len(pdf))
text = "\n".join(pdf)

283


In [18]:
print(text.find("the"))

130


In [8]:
len(text)

1340715

In [29]:
text[start:start+2000]

'project work, HOCHTIEF therefore adopts\n outcomes of the policies, or         a 360-degree, solution-driven approach, taking the en-    In our project activities, we make the best possible al-\n risks) relating to BICC, as no\n policy for the non-financial as-     tire project life cycle into account. As part of this ap- lowance for social and environmental impacts from an\n pects has been or is pursued\n for this part of the Group. No       proach, we also seek to balance economic, environ-        early stage. Wherever practicable, aspects such as en-\n non-financial information has        mental, and social dimensions (see chart). This calls for vironmental protection as well as occupational safety\n been reported to HOCHTIEF.\n                                      dialog with the various stakeholders, which we con-       and health are integrated right from the planning\n                                     duct openly and in a spirit of partnership. It is our ap-  stage.2) During

# ghostscript

In [58]:
f = f"/mnt/d/data1/reports/{f}"
cmd = f"gs -q -dLastPage=3 -dNODISPLAY -dSAFER -dDELAYBIND -dWRITESYSTEMDICT -dSIMPLE -c save -f ps2ascii.ps {f} -c quit >temp.txt"
cmd

'gs -q -dLastPage=3 -dNODISPLAY -dSAFER -dDELAYBIND -dWRITESYSTEMDICT -dSIMPLE -c save -f ps2ascii.ps /mnt/d/data1/reports/0dbd0328-6b3f-46fd-86dd-5dd09923adb4.pdf -c quit >temp.txt'

# spacy

In [9]:
# python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")

In [34]:
text = 'reduce emissions in xyz division 10% by 2025'
doc = nlp(text)
print([(ent.text, ent.label_, ent.start, ent.end) for ent in doc.ents])

[('xyz', 'GPE', 3, 4), ('10%', 'PERCENT', 5, 7), ('2025', 'DATE', 8, 9)]


In [27]:
text = 'he walked 300km on 3rd June'
doc = nlp(text)
print([(ent.text, ent.label_, ent.start, ent.end) for ent in doc.ents])

[('300km', 'QUANTITY', 2, 4), ('3rd June', 'DATE', 5, 7)]


In [ ]:
"""            SPACY                 DUCKLING
start/end      words                 characters
quantity/units together              separate
percentage     entity                number
time           raw text              plus datetime, from/to
org            yes                   no
"""

In [ ]:
sbd = nlp.add_pipe('sentencizer')
doc = nlp(text)
sents = list(doc.sents)
print(sents)

# duckling

In [14]:
# docker run -d --name duckling --restart=always --port 8000:8000 rasa/duckling
import requests
def duck(text):
    return requests.post("http://localhost:8000/parse", data=dict(text=text)).json()

In [18]:
duck("300km")

[{'body': '300km',
  'start': 0,
  'value': {'value': 300, 'type': 'value', 'unit': 'kilometre'},
  'end': 5,
  'dim': 'distance',
  'latent': False}]

In [26]:
text = 'reduce emissions in xyz division 10% by 2025'
doc = duck(text)
doc

[{'body': '10',
  'start': 33,
  'value': {'value': 10, 'type': 'value'},
  'end': 35,
  'dim': 'number',
  'latent': False},
 {'body': 'by 2025',
  'start': 37,
  'value': {'values': [{'to': {'value': '2025-01-01T00:00:00.000-08:00',
      'grain': 'second'},
     'from': {'value': '2022-06-17T06:36:45.599-07:00', 'grain': 'second'},
     'type': 'interval'}],
   'to': {'value': '2025-01-01T00:00:00.000-08:00', 'grain': 'second'},
   'from': {'value': '2022-06-17T06:36:45.599-07:00', 'grain': 'second'},
   'type': 'interval'},
  'end': 44,
  'dim': 'time',
  'latent': False}]